<a href="https://colab.research.google.com/github/erelin6613/ML/blob/master/google_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Flatten, SimpleRNN, Reshape
import gensim.downloader as api
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [0]:
def cat_to_numeric(category):
    if category=='LIFE_ARTS':
        return 1
    if category=='CULTURE':
        return 2
    if category=='SCIENCE':
        return 3
    if category=='STACKOVERFLOW':
      return 4
    if category=='TECHNOLOGY':
      return 5

In [3]:
train_data = pd.read_csv('/content/train.csv', encoding='utf-8')
final_test_data = pd.read_csv('/content/test.csv', encoding='utf-8')

to_drop_train = ['question_user_page', 'host', 'url', 'answer_user_page', 
                 'answer_user_name', 'question_user_name', 'question_user_page']
#train_data.set_index('qa_id', inplace=True)
train_data = train_data.drop(to_drop_train, axis=1)
#print(train_data['category'].unique())
train_data['categoty_normalized'] = train_data['category'].apply(cat_to_numeric)
print(train_data.columns)
#print(train_data['categoty_normalized'])
train_data.drop('category', axis=1, inplace=True)
#train_data['category_encoded'] = to_categorical(train_data['categoty_normalized'],
#                                  num_classes=5)
train_cols = final_test_data.columns
target_cols = set(train_data.columns).difference(set(final_test_data.columns))
train_cols = set(train_data.columns) - target_cols
target_cols = list(target_cols)
train_cols = list(train_cols)

print(train_cols)
print(target_cols)

Index(['qa_id', 'question_title', 'question_body', 'answer', 'category',
       'question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'ans

In [4]:
X_train, X_test, y_train, y_test = train_test_split(train_data.loc[:, train_cols], train_data.loc[:, target_cols], test_size=0.25)
text_cols = ['question_title', 'question_body', 'answer']
tokenizer = Tokenizer()
for col in text_cols:
  tokenizer.fit_on_texts(train_data[col])
print(X_train.columns)
for col in text_cols:
  X_train[col+'_tokenized'] = tokenizer.texts_to_sequences(X_train[col])
  X_train.drop([col], inplace=True, axis=1)
  X_test[col+'_tokenized'] = tokenizer.texts_to_sequences(X_test[col])
  X_test.drop([col], inplace=True, axis=1)
  #X_train[col+'_normalized'] = pad_sequences(X_train[col+'_tokenized'], maxlen=750)
  #print(X_train[col+'_tokenized'])
"""
for col in text_cols:
  X_test.loc[:, col+'_tokenized'] = pd.Series([arr for arr in X_train[col+'_tokenized']])
  X_test.loc[:, col+'_tokenized'] = pd.Series(pad_sequences(X_train[col+'_tokenized'], maxlen=750, padding='post'))
  #arr_question_tittle = [arr for arr in X_train['question_title_tokenized']]
  #pads_question_tittle = pad_sequences(arr_question_tittle, maxlen=750, padding='post')
  print(X_test.loc[:, col+'_tokenized'])

for column, row in X_train.iterrows():
  print(pad_sequences([row[text_cols[0]+'_tokenized'], row[text_cols[1]+'_tokenized'], row[text_cols[2]+'_tokenized']], maxlen=700))
  X_train.loc[row['qa_id'], col+'_normalized'] =  pad_sequences([row[text_cols[0]+'_tokenized'], row[text_cols[1]+'_tokenized'], row[text_cols[2]+'_tokenized']], maxlen=700)
  #print(X_train.loc[row['qa_id'], col+'_normalized'])
  break
    #X_train.drop(col+'_tokenized', inplace=True)
"""

print(X_train['question_body_tokenized'])


Index(['qa_id', 'answer', 'question_body', 'question_title'], dtype='object')
2644    [4, 1053, 32, 3849, 16, 1638, 2138, 2, 496, 29...
422     [33, 4, 71, 223, 2, 207, 5476, 5715, 51, 368, ...
1651    [4, 971, 741, 12, 232, 165, 48, 1, 676, 1047, ...
5836    [4, 71, 1053, 32, 2357, 72, 2191, 72, 1310, 72...
3688    [119, 244, 446, 39, 5, 306, 51, 517, 205, 8, 1...
                              ...                        
3919    [4, 71, 64, 8, 456, 1179, 6, 8, 154, 4, 71, 27...
2727    [4, 20, 32, 2732, 13, 2732, 13149, 27889, 4075...
2082    [1481, 1022, 73, 7, 3, 1954, 8, 15740, 353, 96...
5658    [1, 4005, 15, 29, 2424, 1861, 56, 769, 6767, 8...
996     [590, 1, 8309, 13791, 7, 4463, 1, 9861, 5, 109...
Name: question_body_tokenized, Length: 4559, dtype: object


In [0]:
print(tokenizer.word_index)

{'the': 1, 'to': 2, 'a': 3, 'i': 4, 'is': 5, 'and': 6, 'of': 7, 'in': 8, 'that': 9, 'it': 10, 'you': 11, 'this': 12, 'for': 13, 'gt': 14, 'on': 15, 'with': 16, 'be': 17, 'if': 18, 'lt': 19, 'have': 20, '1': 21, 'as': 22, 'not': 23, 'are': 24, 'can': 25, 'but': 26, 'or': 27, '0': 28, 'my': 29, 'from': 30, '2': 31, 'an': 32, 'so': 33, 'at': 34, 'your': 35, 'do': 36, 'will': 37, 'what': 38, 'there': 39, 'how': 40, 'would': 41, 'by': 42, 'use': 43, 'one': 44, 'which': 45, 'when': 46, 'like': 47, 'all': 48, 'get': 49, 'x': 50, 'some': 51, 'then': 52, 'any': 53, 'using': 54, 'they': 55, 'just': 56, 'was': 57, 'more': 58, 'has': 59, '3': 60, 'should': 61, 'other': 62, 'want': 63, 'new': 64, 'we': 65, "i'm": 66, 'time': 67, 'no': 68, 'need': 69, 'only': 70, 'am': 71, 'amp': 72, 'out': 73, 'way': 74, 'about': 75, 'also': 76, 'n': 77, 'up': 78, "'": 79, 'code': 80, 'c': 81, 'file': 82, 'me': 83, 'could': 84, 'does': 85, "it's": 86, "don't": 87, 'id': 88, 'where': 89, 'them': 90, 'make': 91, 'sam

In [25]:
#word_vectors = api.load("glove-wiki-gigaword-100")
X_train.drop('qa_id', axis=1)
maxlen = 1000
X_train_stacked = []

for column, row in X_train.iterrows():
  items = []
  #print(X_train.loc[row['qa_id'], 'question_title_tokenized'])
  #print(X_train.loc[row['qa_id'], 'question_body_tokenized'])
  #print(X_train.loc[row['qa_id'], 'answer_tokenized'])
  #print(X_train.drop('qa_id', axis=1).columns)
  for each in X_train.drop('qa_id', axis=1).columns:
    if len(X_train.loc[row['qa_id'], each]) > 1000:
      item_to_stack = X_train.loc[row['qa_id'], each][:1000]
    else:
      item_to_stack = X_train.loc[row['qa_id'], each]
      while len(X_train.loc[row['qa_id'], each]) < 1000:
        item_to_stack.append(0)
    items.append(item_to_stack)
  #print(items)
  X_train_stacked.append(np.stack(items))
print(X_train_stacked)

KeyError: ignored

In [6]:
model = Sequential()

model.add(Reshape((None, None, 100), input_shape=X_train.loc[0, :].shape))

#model.add(word_vectors.get_keras_embedding(train_embeddings=True))
model.add(word_vectors.get_keras_embedding(train_embeddings=True))
#model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.15))

model.add(Dense(32, activation='relu'))
#model.add(SimpleRNN(30))
model.compile(optimizer='Adam', loss='mean_squared_error')
model.summary()

ValueError: ignored

In [0]:
model.fit(X_train, y_train)
model.save('google_data.model')
#Index(['qa_id', 'question_body', 'category', 'answer', 'question_user_name',
#'answer_user_name', 'question_title'],
#dtype='object')

ValueError: ignored